In [3]:
search_for = 126
start_from = 0
threads = 10
thread = 5
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0021729469299316406
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 375
all cases: 956096141
type: [1, 1, 1, 1, 126] 126
cases of this type: 252047376
100000 23.51637795319532
200000 38.130586808876835
300000 48.23413167588215
400000 52.311845681155724
500000 54.04502410632241
600000 53.707047176838216
700000 50.073700505474505
800000 54.09678578629313
900000 53.28110152655991
1000000 42.482397345978356
1100000 36.001586835813626
1200000 57.90019752005816
1300000 62.995329126526684
1400000 63.151631357915086
1500000 59.92946238017589
1600000 61.41523065406779
1700000 56.268677047132954
1800000 47.94716002895922
1900000 53.536468492852514
2000000 52.699392728552596
2100000 28.535419764155968
2200000 51.19088208311582
2300000 51.79754852414033
2400000 61.78909995570058
2500000 47.86573357844492
2600000 40.91167305299616
2700000 49.31208626064783
2800000 51.36258666857962
2900000 59.81751191001336
3000000 40.01

30000000 74.27548950367712
30100000 78.4319144962514
30200000 78.46786262724683
30300000 31.168749986695158
30400000 82.15117374898018
30500000 82.57123605982072
30600000 77.30285526092189
30700000 78.47255608720313
30800000 74.48792716282107
30900000 72.31267461487022
31000000 55.92565339285874
31100000 41.45787500104549
31200000 70.83291929108596
31300000 72.12459574413043
31400000 75.58817639017748
31500000 72.77415249149436
31600000 77.1569636256879
31700000 79.39073183088149
31800000 41.76164092140123
31900000 74.9637811711643
32000000 76.56484752489537
32100000 77.73158925232451
32200000 71.48653371225976
32300000 34.92803749169606
32400000 71.05477344605265
32500000 73.27448169092311
32600000 73.81932979574863
32700000 70.36554134780968
32800000 74.45209997486992
32900000 66.00853633412925
33000000 54.832357216759206
33100000 39.90183696801647
33200000 65.79954022033287
33300000 68.3146209041468
33400000 68.924257692944
33500000 74.1245860483765
33600000 73.32639874546693
337000

60400000 71.26779310369231
60500000 48.33843690436947
60600000 51.208954695174796
60700000 64.49405325321447
60800000 62.37387067974897
60900000 65.81869772477233
61000000 55.767933678645605
61100000 34.41801565928147
61200000 65.0138076662826
61300000 65.15742520658658
61400000 64.50856848466226
61500000 63.59832220156055
61600000 32.3438947346829
61700000 69.39883718094715
61800000 69.00844964993198
61900000 67.49181367922496
62000000 73.49166514902424
62100000 79.64591276066855
62200000 73.49088255887409
62300000 75.77158633594068
62400000 72.73668425648754
62500000 59.489701938629494
62600000 41.17442403822866
62700000 67.25679990762477
62800000 68.95298882504824
62900000 67.15284677786336
63000000 55.98293450050516
63100000 34.14967485317653
63200000 65.05130884531219
63300000 66.90853508742892
63400000 65.8379661060641
63500000 59.98265908172943
63600000 40.512010476142514
63700000 71.63660832300756
63800000 73.29341480605063
63900000 72.79281492447515
64000000 78.69863564031486


90700000 64.28745957782837
90800000 26.968748897006474
90900000 63.833020887005816
91000000 56.85901912156659
91100000 30.288424976887992
91200000 61.44448428441741
91300000 49.36959170069038
91400000 43.988570735320096
91500000 60.19418079724278
91600000 63.45826958533797
91700000 64.15498557859323
91800000 72.64865358228306
91900000 69.3138119604981
92000000 69.72227502791613
92100000 66.20065513103746
92200000 63.39134978344189
92300000 65.32008417912506
92400000 64.28470253286831
92500000 60.65411336754872
92600000 60.14055814472612
92700000 58.849954213268894
92800000 26.50744765943936
92900000 59.73488897541491
93000000 54.315142446632905
93100000 30.087653903881904
93200000 58.703283135058804
93300000 45.62579624396179
93400000 48.06576279691802
93500000 59.670469372782584
93600000 63.253744255944966
93700000 63.59796002326782
93800000 64.0965977772999
93900000 65.55956252329477
94000000 66.90692178654604
94100000 70.56362284302834
94200000 71.68542672676801
94300000 67.72413288

120100000 83.88076790893639
120200000 76.32284480808038
120300000 78.43136645859796
120400000 71.23002841646449
120500000 74.85563538730311
120600000 73.7370661489971
120700000 72.16828801420212
120800000 71.31490473953095
120900000 75.54005002482967
121000000 38.953399282506474
121100000 11.756141729841255
121200000 64.42811560143468
121300000 69.91215576043395
121400000 71.41519877636948
121500000 75.93578250977804
121600000 75.52102847834645
121700000 76.44256976223197
121800000 75.80347597885101
121900000 77.27936415712928
122000000 81.48739230428826
122100000 25.388459791090916
122200000 6.462789442145387
122300000 6.369159010394072
122400000 7.162939131037383
122500000 6.311233557091387
122600000 7.018662217045987
122700000 7.106612919278358
122800000 6.49825426922804
122900000 6.454670455816704
123000000 6.771124572861699
123100000 6.72628676223923
123200000 6.334089934137498
123300000 6.854397654837187
123400000 6.329784016641313
123500000 6.393454623905864
123600000 6.10818294

...

884400000 2.834239033052305
884500000 1.8931300775692625
884600000 3.033767739370209
884700000 2.298787411400008
884800000 3.1996425714346532
884900000 2.0760315341872433
885000000 2.9540161140473775
885100000 2.090468523326521
885200000 2.8353396499761883
885300000 2.576303776267056
885400000 1.9593074152775982
885500000 2.8378590878886683
885600000 1.7612148082135617
885700000 2.9109327236598537
885800000 1.582354825889653
885900000 3.1208149404060195
886000000 1.7401416746251333
886100000 2.410974360943812
886200000 2.1385890339682514
886300000 2.3680715946107393
886400000 3.4502369296320143
886500000 1.720219510171171
886600000 1.1384552795790228
886700000 0.396824833240056
886800000 0.47580782493893303
886900000 0.5034316592983875
887000000 0.5004957114134967
887100000 0.38093875896857377
887200000 0.44802502496973473
887300000 0.3975770990177592
887400000 0.5028324173614032
887500000 0.5083923818277358
887600000 0.5085773784953435
887700000 2.4921401905237217
887800000 2.03744758

909000000 0.3920675734448029
909100000 0.3084285716012915
909200000 0.2074354098452131
best so far: 0
type: [1, 3, 6, 7, 1] 126
cases of this type: 37044
best so far: 0
type: [1, 3, 7, 1, 6] 126
cases of this type: 190512
909300000 0.6732396674864378
909400000 0.37757178556679616
best so far: 0
type: [1, 3, 7, 2, 3] 126
cases of this type: 95256
909500000 0.34224863599373107
best so far: 0
type: [1, 3, 7, 3, 2] 126
cases of this type: 63504
909600000 0.3138579211867187
best so far: 0
type: [1, 3, 7, 6, 1] 126
cases of this type: 31752
best so far: 0
type: [1, 3, 14, 1, 3] 126
cases of this type: 47628
909700000 0.30608233391995165
best so far: 0
type: [1, 3, 14, 3, 1] 126
cases of this type: 15876
best so far: 0
type: [1, 3, 21, 1, 2] 126
cases of this type: 21168
best so far: 0
type: [1, 3, 21, 2, 1] 126
cases of this type: 10584
best so far: 0
type: [1, 3, 42, 1, 1] 126
cases of this type: 5292
best so far: 0
type: [1, 6, 1, 1, 21] 126
cases of this type: 1166886
909800000 0.70351751

920600000 1.0561158442060747
920700000 0.8576981715092089
920800000 1.0378507142530686
920900000 0.9902310381752492
921000000 1.1231399642287307
921100000 1.3150497792865838
921200000 0.8554821894049466
921300000 1.3051867836912134
921400000 0.813059460460533
921500000 0.919954885336435
921600000 1.0855883512213633
921700000 0.7479614311008267
921800000 1.0645446617512029
921900000 0.773715000847639
922000000 0.7723531311894125
922100000 0.2279291081717968
922200000 0.2314138073751536
922300000 0.18414562629037962
922400000 0.15510321936552526
922500000 0.169984009844425
922600000 0.21506968477030464
922700000 0.1799355195590238
922800000 0.21347724141847887
922900000 0.14629479579072527
923000000 0.14322479512504938
923100000 0.14767733824442558
923200000 0.14515823292459118
923300000 0.5430688275804777
923400000 0.9797364877307011
923500000 0.9306478567521141
923600000 0.695046991686388
923700000 0.451150184993881
923800000 0.8558606733556356
923900000 0.4496714363777757
924000000 0.

944400000 0.11892719898994367
944500000 0.048448794136010276
944600000 0.15490115273600222
944700000 0.16630309934411314
best so far: 0
type: [2, 3, 1, 3, 7] 126
cases of this type: 194481
944800000 0.09941492037284508
944900000 0.07086661970151266
best so far: 0
type: [2, 3, 1, 7, 3] 126
cases of this type: 83349
945000000 0.06698195218910309
best so far: 0
type: [2, 3, 1, 21, 1] 126
cases of this type: 27783
best so far: 0
type: [2, 3, 3, 1, 7] 126
cases of this type: 64827
945100000 0.05694931901154916
best so far: 0
type: [2, 3, 3, 7, 1] 126
cases of this type: 9261
best so far: 0
type: [2, 3, 7, 1, 3] 126
cases of this type: 11907
best so far: 0
type: [2, 3, 7, 3, 1] 126
cases of this type: 3969
best so far: 0
type: [2, 3, 21, 1, 1] 126
cases of this type: 1323
best so far: 0
type: [2, 7, 1, 1, 9] 126
cases of this type: 45927
best so far: 0
type: [2, 7, 1, 3, 3] 126
cases of this type: 15309
945200000 0.09942676243812243
best so far: 0
type: [2, 7, 1, 9, 1] 126
cases of this type

best so far: 0
type: [3, 7, 2, 1, 3] 126
cases of this type: 2268
best so far: 0
type: [3, 7, 2, 3, 1] 126
cases of this type: 756
best so far: 0
type: [3, 7, 3, 1, 2] 126
cases of this type: 1008
best so far: 0
type: [3, 7, 3, 2, 1] 126
cases of this type: 504
best so far: 0
type: [3, 7, 6, 1, 1] 126
cases of this type: 252
best so far: 0
type: [3, 14, 1, 1, 3] 126
cases of this type: 1134
best so far: 0
type: [3, 14, 1, 3, 1] 126
cases of this type: 378
best so far: 0
type: [3, 14, 3, 1, 1] 126
cases of this type: 126
best so far: 0
type: [3, 21, 1, 1, 2] 126
cases of this type: 336
best so far: 0
type: [3, 21, 1, 2, 1] 126
cases of this type: 168
best so far: 0
type: [3, 21, 2, 1, 1] 126
cases of this type: 84
best so far: 0
type: [3, 42, 1, 1, 1] 126
cases of this type: 42
best so far: 0
type: [6, 1, 1, 1, 21] 126
cases of this type: 194481
955400000 0.008711144120889903
955500000 0.006677900168957312
best so far: 0
type: [6, 1, 1, 3, 7] 126
cases of this type: 64827
best so far: 0